In [2]:
import pandas as pd
import numpy as np

In [3]:
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Ram\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    archspec-0.1.1             |     pyh9f0ad1d_0          25 KB  conda-forge
    conda-4.8.4                |   py37hc8dfbb8_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  archspec           conda-forge/noarch::archspec-0.1.1-pyh9f0ad1d_0

The following packages will be UPDATED:

  conda                                4.8.3-py37hc8dfbb8_1 --> 4.8.4-py37hc8dfbb8_0




archspec-0.1.1       | 25 KB     |            |   0% 
archspec-0.1.1       | 25 KB     | ######3    |  64% 
archspec-0.1.1       | 25 KB     | ########## | 100% 

conda-4.8.4

In [4]:
import requests

Using this website to generate all the zipcodes in Suffolk County (home of Boston)

In [36]:
url = 'https://www.zip-codes.com/city/ma-boston.asp'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[2]
print(df)
df.to_csv('Boston_postal.csv')

                 0         1        2           3  \
0         ZIP Code      Type   County  Population   
1   ZIP Code 02108  Standard  Suffolk        3825   
2   ZIP Code 02109  Standard  Suffolk        3771   
3   ZIP Code 02110  Standard  Suffolk        1733   
4   ZIP Code 02111  Standard  Suffolk        7383   
5   ZIP Code 02112  P.O. Box  Suffolk           0   
6   ZIP Code 02113  Standard  Suffolk        6915   
7   ZIP Code 02114  Standard  Suffolk       11999   
8   ZIP Code 02115  Standard  Suffolk       28441   
9   ZIP Code 02116  Standard  Suffolk       20628   
10  ZIP Code 02117  P.O. Box  Suffolk           0   
11  ZIP Code 02118  Standard  Suffolk       26498   
12  ZIP Code 02119  Standard  Suffolk       25346   
13  ZIP Code 02120  Standard  Suffolk       15181   
14  ZIP Code 02121  Standard  Suffolk       25978   
15  ZIP Code 02122  Standard  Suffolk       23479   
16  ZIP Code 02123  P.O. Box  Suffolk           0   
17  ZIP Code 02124  Standard  Suffolk       47

In [38]:
newDF = df.drop(columns = [1,3,4])

In [39]:
newDF.columns = newDF.iloc[0]

In [40]:
newDF = newDF.drop([0,0])

In [71]:
newDF['ZIP Code'] = newDF['ZIP Code'].str[9:]

In [ ]:
newDF["ZIP Code"] = newDF["ZIP Code"].astype(str).astype(int)

In [74]:
filename = 'US_Zip_Codes.csv'
df2 = pd.read_csv('US_Zip_Codes.csv')

In [75]:
df2 = df2.rename(columns={"ZIP": "ZIP Code"})

In [76]:
df2.head()

,ZIP Code,LAT,LNG
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158345,-66.932911
4,610,18.295366,-67.125135


In [65]:
fullDF = newDF.merge(df2, on='ZIP Code', how='left')

In [77]:
nonNA = fullDF.dropna()

In [81]:
nonNA.head()

,ZIP Code,County,LAT,LNG
0,2108,Suffolk,42.357768,-71.064858
1,2109,Suffolk,42.367032,-71.050493
2,2110,Suffolk,42.361962,-71.047846
3,2111,Suffolk,42.350518,-71.059077
5,2113,Suffolk,42.365331,-71.055233


In [82]:
df3 = pd.read_csv('neighborhood_data.csv')

In [87]:
df3 = df3.rename(columns={"Zip Code": "ZIP Code"})
df3.head()

,OBJECTID,Name,Acres,Neighborhood_ID,SqMiles,ShapeSTArea,ShapeSTLength,ZIP Code
0,27,Roslindale,1605.568238,15,2.51,6.993827e+07,53563.912600,2131
1,28,Jamaica Plain,2519.245394,11,3.94,1.097379e+08,56349.937160,2130
2,29,Mission Hill,350.853564,13,0.55,1.528312e+07,17918.724110,2120
3,30,Longwood,188.611947,28,0.29,8.215904e+06,11908.757150,2115
4,31,Bay Village,26.539839,33,0.04,1.156071e+06,4650.635493,2116


In [84]:
df3.dtypes

OBJECTID             int64
Name                object
Acres              float64
Neighborhood_ID      int64
SqMiles            float64
ShapeSTArea        float64
ShapeSTLength      float64
Zip Code             int64
dtype: object

In [88]:
fullDF2 = fullDF.merge(df3, on='ZIP Code', how='left')

In [95]:
fullDF2.head()

,ZIP Code,LAT,LNG,Name,Acres,Neighborhood_ID,SqMiles,ShapeSTArea,ShapeSTLength
0,2108,42.357768,-71.064858,Beacon Hill,200.156904,30.0,0.31,8.718800e+06,14303.829020
1,2109,42.367032,-71.050493,Harbor Islands,824.888658,22.0,1.29,3.593201e+07,92482.183570
2,2110,42.361962,-71.047846,NaN,NaN,NaN,NaN,NaN,NaN
3,2111,42.350518,-71.059077,Leather District,15.639908,27.0,0.02,6.812717e+05,3237.140537
4,2111,42.350518,-71.059077,Chinatown,76.324410,26.0,0.12,3.324678e+06,9736.590413


In [105]:
#fullDF2 = fullDF2.drop(columns=['OBJECTID', 'County', 'Neighborhood_ID'])
fullDF2 = fullDF2.drop(columns=['Neighborhood_ID'])

In [106]:
finalDF = fullDF2.dropna()

In [107]:
finalDF.reset_index(drop=True, inplace=True)

In [108]:
finalDF.head()

,ZIP Code,LAT,LNG,Name,Acres,SqMiles,ShapeSTArea,ShapeSTLength
0,2108,42.357768,-71.064858,Beacon Hill,200.156904,0.31,8.718800e+06,14303.829020
1,2109,42.367032,-71.050493,Harbor Islands,824.888658,1.29,3.593201e+07,92482.183570
2,2111,42.350518,-71.059077,Leather District,15.639908,0.02,6.812717e+05,3237.140537
3,2111,42.350518,-71.059077,Chinatown,76.324410,0.12,3.324678e+06,9736.590413
4,2113,42.365331,-71.055233,North End,126.910439,0.20,5.527506e+06,16177.826820


Finally, the required neighborhood dataframes have been assembled properly. For this project, I am following the similar concept from the previous project from week 3 where I would use the following values: latitude, longitude, and for identifying purposes, the name of the neighborhood. In case it was difficult to tell, I initially parsed through a list of zip codes in Boston from a website, and then I combined that list of zip codes with a csv file that had the corresponding latitude and longitude with the Zip code being the key. I found this file from a github: https://gist.github.com/erichurst/7882666. 
Once I had this, I combined this with a csv file of the Boston neighborhoods based on Zip codes (which I created myself), and used the Zip as the key once again. This was going to be my final dataframe for the project (finalDF). 

**Creating Map of Boston**

In [115]:
address = 'Boston, MA'

geolocator = Nominatim(user_agent="Boston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Boston are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Boston are 42.3602534, -71.0582912.


In [125]:
# Creating a map of Boston
map_Boston = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(finalDF['LAT'], finalDF['LNG'], finalDF["Name"]):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Boston)  
    
map_Boston

As you can see from the map produced, it only displays 19 of the 25 neighborhoods. This is because the neighborhoods from the csv files repeat the same latitude and longitudes for the location since some of them share the same zip codes. As a result, the map relied on the larger area defined neighborhoods to display their names instead. 

In [126]:
CLIENT_ID = 'yer' # your Foursquare ID
CLIENT_SECRET = 'yoink' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [128]:
neighborhood_latitude = finalDF.loc[0, 'LAT'] # neighborhood latitude value
neighborhood_longitude = finalDF.loc[0, 'LNG'] # neighborhood longitude value
neighborhood_name = finalDF.loc[0, 'Name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Beacon Hill are 42.357768, -71.064858.


In [129]:
LIMIT = 100
radius = 800
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LQSN2LMHVZWTW4Y50YZSPI3QGXFUWWZARIWV213WFE2XJBB4&client_secret=RRD2SQQNAXTRGPRHJN0UX3CA5JNIIBPUGS4PLEK2YI4RNTRH&v=20180604&ll=42.357768,-71.064858&radius=800&limit=100'

In [132]:
results = requests.get(url).json()
#results

In [133]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [134]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()
#nearby_venues

C:\Users\Ram\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Boston Common,Park,42.355487,-71.064882
1,Boston Athenaeum,Library,42.357481,-71.061838
2,Frog Pond,Lake,42.356134,-71.065672
3,Florina,Pizza Place,42.359375,-71.063219
4,No. 9 Park,French Restaurant,42.357540,-71.063193


In [135]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [136]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [138]:
Boston_venues = getNearbyVenues(names=finalDF['Name'], latitudes=finalDF['LAT'], longitudes=finalDF['LNG'])

Beacon Hill
Harbor Islands
Leather District
Chinatown
North End
West End
Longwood
Back Bay
Fenway
Bay Village
Roxbury
Mission Hill
Dorchester
Mattapan
South Boston Waterfront
South Boston
East Boston
Charlestown
Jamaica Plain
Roslindale
South End
West Roxbury
Allston
Brighton
Hyde Park


In [141]:
print(Boston_venues.shape)
Boston_venues.tail(5)

(986, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
981,Hyde Park,42.255083,-71.12922,Rite Aid,42.253319,-71.125626,Pharmacy
982,Hyde Park,42.255083,-71.12922,Shaw's,42.251787,-71.126090,Grocery Store
983,Hyde Park,42.255083,-71.12922,MBTA Hyde Park,42.254561,-71.125591,Train Station
984,Hyde Park,42.255083,-71.12922,McDonald's,42.252866,-71.124458,Fast Food Restaurant
985,Hyde Park,42.255083,-71.12922,New York Fried Chicken,42.255844,-71.123624,Fried Chicken Joint


In [146]:
Boston_venues.head(50)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Beacon Hill,42.357768,-71.064858,Frog Pond,42.356134,-71.065672,Lake
1,Beacon Hill,42.357768,-71.064858,Boston Common,42.355487,-71.064882,Park
2,Beacon Hill,42.357768,-71.064858,Boston Athenaeum,42.357481,-71.061838,Library
3,Beacon Hill,42.357768,-71.064858,No. 9 Park,42.357540,-71.063193,French Restaurant
4,Beacon Hill,42.357768,-71.064858,Florina,42.359375,-71.063219,Pizza Place
5,Beacon Hill,42.357768,-71.064858,Zen Japanese Grill & Sushi Bar,42.358020,-71.062531,Sushi Restaurant
6,Beacon Hill,42.357768,-71.064858,Museum of African American History,42.360058,-71.065287,History Museum
7,Beacon Hill,42.357768,-71.064858,Grotto,42.359915,-71.062807,Italian Restaurant
8,Beacon Hill,42.357768,-71.064858,Nichols House Museum,42.358342,-71.066019,Museum
9,Beacon Hill,42.357768,-71.064858,Union Club of Boston,42.357411,-71.062702,Restaurant


In [147]:
Boston_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(Boston_venues['Venue Category'].unique())))

There are 185 uniques categories.


In [148]:
Boston_onehot = pd.get_dummies(Boston_venues[['Venue Category']], prefix="", prefix_sep="")
Boston_onehot.drop(['Neighborhood'], axis=1, inplace = True)
Boston_onehot.insert(loc=0, column='Neighborhood', value=Boston_venues['Neighborhood'] )
Boston_onehot.shape

(986, 185)

In [149]:
Boston_grouped = Boston_onehot.groupby('Neighborhood').mean().reset_index()
Boston_grouped.shape

(25, 185)

In [150]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [155]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Boston_grouped['Neighborhood']

for ind in np.arange(Boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(19)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,Rental Car Location,Liquor Store,Bar,Plaza,Pizza Place,Smoke Shop,Chinese Restaurant,Donut Shop,Thrift / Vintage Store,Fast Food Restaurant
1,Back Bay,Café,Coffee Shop,Falafel Restaurant,American Restaurant,Sushi Restaurant,Sandwich Place,Italian Restaurant,Fast Food Restaurant,Burrito Place,Shipping Store
2,Bay Village,Coffee Shop,American Restaurant,Spa,Hotel,Clothing Store,Seafood Restaurant,Department Store,Gym,Ice Cream Shop,Juice Bar
3,Beacon Hill,Coffee Shop,Italian Restaurant,Pizza Place,American Restaurant,Hotel,Plaza,Sushi Restaurant,New American Restaurant,French Restaurant,Clothing Store
4,Brighton,Pizza Place,Chinese Restaurant,Bank,Bakery,Bus Station,Pub,Dry Cleaner,Café,Coffee Shop,Mobile Phone Shop
5,Charlestown,Pizza Place,Convenience Store,Gym,Dry Cleaner,Martial Arts School,Mobile Phone Shop,Monument / Landmark,Music Venue,National Park,Café
6,Chinatown,Asian Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Sandwich Place,Sushi Restaurant,Theater,Café,Dive Bar,Restaurant
7,Dorchester,Zoo Exhibit,Fast Food Restaurant,Shopping Mall,Men's Store,Playground,Pharmacy,Caribbean Restaurant,Nightclub,Donut Shop,Farmers Market
8,East Boston,Airport Service,Zoo Exhibit,Donut Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
9,Fenway,Café,Coffee Shop,Falafel Restaurant,American Restaurant,Sushi Restaurant,Sandwich Place,Italian Restaurant,Fast Food Restaurant,Burrito Place,Shipping Store
